In [3]:
import requests
import csv

def query_wikidata():
    #meteorites in United States, Mexico, or Canada
    sparql_query = """
    SELECT ?meteorite ?meteoriteLabel ?discovery_time
    WHERE
    {
        ?meteorite wdt:P31 wd:Q60186;  # instance of meteorite
                   wdt:P17 ?country;    # located in country
                   wdt:P575 ?discovery_time.  # time of discovery
        FILTER (?country IN (wd:Q30, wd:Q96, wd:Q16))  # United States, Mexico, Canada
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """

    api_url = "https://query.wikidata.org/sparql"

    response = requests.get(api_url, params={'format': 'json', 'query': sparql_query})

    if response.status_code == 200:
        return response.json()['results']['bindings']
    else:
        print("Error fetching data from Wikidata:", response.text)
        return None

def save_to_csv(data):
    csv_file = 'meteorites_wikidata.csv'

    # Write to CSV
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # header
        writer.writerow(["Meteorite ID", "Meteorite Name", "Discovery Time"])
        # data rows
        for item in data:
            meteorite_id = item['meteorite']['value'].split('/')[-1]
            meteorite_name = item['meteoriteLabel']['value']
            discovery_time = item['discovery_time']['value']
            writer.writerow([meteorite_id, meteorite_name, discovery_time])

def main():
    # Query
    result = query_wikidata()
    if result:
        # CSV
        save_to_csv(result)
        print("Data saved to meteorites_wikidata.csv")
    else:
        print("Failed to fetch data from Wikidata")

if __name__ == "__main__":
    main()


Data saved to meteorites_wikidata.csv


In [4]:
#geojson component

import csv
import json

def create_geojson(input_file, output_file):
    features = []

    with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:

            meteorite_id = row['Meteorite ID']
            meteorite_name = row['Meteorite Name']
            country = row['Country']
            state = row['State']
            year = row['Year']
            coordinates = [float(coord) for coord in row['GeoLocation'].split(',')]
            
            feature = {
                "type": "Feature",
                "properties": {
                    "Meteorite ID": meteorite_id,
                    "Meteorite Name": meteorite_name,
                    "Country": country,
                    "State": state,
                    "Year": year
                },
                "geometry": {
                    "type": "Point",
                    "coordinates": coordinates
                }
            }
            features.append(feature)

    geojson_data = {
        "type": "FeatureCollection",
        "features": features
    }

    with open(output_file, 'w') as jsonfile:
        json.dump(geojson_data, jsonfile, indent=4)

input_file = 'cleaned_meteorites_wikidata.csv'
output_file = 'meteorites_wikidata.geojson'

create_geojson(input_file, output_file)
print("GeoJSON data:", output_file)


GeoJSON data: meteorites_wikidata.geojson
